## ML-Assignment 2
Muhammed Talha Gedikli - 19069608

## Import Libraries and Load Data

In [23]:
# Load important libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.datasets import make_regression

from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import Ridge, Lasso

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.svm import LinearSVC
from sklearn.svm import SVC #note that this is non-linear SVM and its default is RBF
from sklearn.datasets import make_regression #function used for generating synthetic datasets
from sklearn.datasets import make_classification, make_blobs

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error, r2_score


# Import data
train_data = pd.read_excel("./asg1_train_test_data/asg1_traindata.xlsx")
test_data = pd.read_excel("./asg1_train_test_data/asg1_testdata_without_prices.xlsx")

test_data_without_prices = pd.read_excel("./asg1_train_test_data/asg1_testdata_without_prices.xlsx")

features_X = ["Year", "Type", "Shift", "km", "Power", "Engine", "Seller"]
features_y = ["Price"]


# Split data into features - prices
X_data = train_data[features_X]
y_data = train_data[features_y]

X_data_without_prices = test_data_without_prices


# Split data into train and test sets
# default is 75% / 25% train-test split
# X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=735)

# X_test = test_data[features_X]
# y_test = test_data[features_y] Y values of test is not given
# y_test = pd.read_csv("./Result.txt", sep=" ", header=None) S

# default 'n' value for knn or cv
N = 5

In [11]:
''' 
# Deal with categorical data
from sklearn.preprocessing import OneHotEncoder

# Find the columns to deal with
object_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(X_test[object_cols]))

# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")
OH_cols_train.columns = OH_encoder.get_feature_names_out(object_cols)
OH_cols_test.columns = OH_encoder.get_feature_names_out(object_cols)

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)
'''



' \n# Deal with categorical data\nfrom sklearn.preprocessing import OneHotEncoder\n\n# Find the columns to deal with\nobject_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]\n\n# Apply one-hot encoder to each column with categorical data\nOH_encoder = OneHotEncoder(handle_unknown=\'ignore\', sparse=False)\nOH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))\nOH_cols_test = pd.DataFrame(OH_encoder.fit_transform(X_test[object_cols]))\n\n# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")\nOH_cols_train.columns = OH_encoder.get_feature_names_out(object_cols)\nOH_cols_test.columns = OH_encoder.get_feature_names_out(object_cols)\n\n# One-hot encoding removed index; put it back\nOH_cols_train.index = X_train.index\nOH_cols_test.index = X_test.index\n\n# Remove categorical columns (will replace with one-hot encoding)\nnum_X_train = X_train.drop(object_cols, axis=1)\nnum_X_test =

## Handle With Categorical Data and Do Feature Scaling

In [12]:
# Deal with categorical data
from sklearn.preprocessing import OneHotEncoder

# Find the columns to deal with
object_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(X_data[object_cols]))
OH_X_without_prices_cols = pd.DataFrame(OH_encoder.fit_transform(X_data_without_prices[object_cols]))


# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")
OH_cols.columns = OH_encoder.get_feature_names_out(object_cols)
OH_X_without_prices_cols.columns = OH_encoder.get_feature_names_out(object_cols)

# One-hot encoding removed index; put it back
OH_cols.index = X_data.index
OH_X_without_prices_cols.index = X_data_without_prices.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_data.drop(object_cols, axis=1)
num_X_without_prices = X_data_without_prices.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_data = pd.concat([num_X_train, OH_cols], axis=1)
OH_X_data_without_prices = pd.concat([num_X_without_prices, OH_X_without_prices_cols], axis=1)

# Split data into train and test sets
# default is 75% / 25% train-test split
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=735)

# Feature scaling
from sklearn.preprocessing import MinMaxScaler

# Set scaler
scaler = MinMaxScaler()

##X_train, X_test, y_train, y_test = train_test_split(X_house, y_house)

# Scale data
X_train_scaled = scaler.fit_transform(X_train) # we use X_train to adjust/fit the scaler
X_test_scaled  = scaler.transform(X_test) #use the same fit found above to transform X_test# Scale data

OH_X_data_scaled = scaler.fit_transform(OH_X_data) # we use X_train to adjust/fit the scaler

## Lineer Regression

In [127]:
# Import required libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score

# Prepare Data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=5)

# Define the model
linr = LinearRegression().fit(X_train, y_train)

kf5 = KFold(n_splits=N, shuffle=True)




# Get and save the r2 values of the model
linr_scores = {
     "r2_train": linr.score(X_train, y_train),
     "r2_test": linr.score(X_test, y_test),
     "r2_cv": cross_val_score(linr, X_train, y_train, cv=N, scoring="r2")
}

# Print the scores
print('R-squared score (training): {:.3f}'
     .format(linr_scores['r2_train']))
print('R-squared score (test): {:.3f}\n'
     .format(linr_scores['r2_test']))
print('CV Score (test): {:.3f}\n'
     .format(linr_scores['r2_cv'].mean()))

R-squared score (training): 0.862
R-squared score (test): 0.890

CV Score (test): 0.864



## Polynomial Regression

In [158]:
# Import required libraries
from sklearn.datasets import make_friedman1
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression #function used for generating synthetic datasets
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

# # Make friedman
# X_F1, y_F1 = make_friedman1(n_samples = 100,
#                            n_features = 7, random_state=0)

# # Prepare data for training and test
# X_train, X_test, y_train, y_test = train_test_split(X_F1, y_F1, random_state = 0)

# Set polynomial features
poly_degree = 1
poly = PolynomialFeatures(degree=poly_degree)
X_F1_poly = poly.fit_transform(OH_X_data)

# Prepare data to make polynomial regression
X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y_data, random_state = 1)
polyr = LinearRegression().fit(X_train, y_train)

kf5 = KFold(n_splits=N, shuffle=True)

polyr_degree = 0
polyr_degree = 0
kf5 = KFold(n_splits=N, shuffle=True)

for this_degree in [1, 2, 3, 4, 5]:
     poly = PolynomialFeatures(degree=this_degree)
     X_F1_poly = poly.fit_transform(OH_X_data)
     # Prepare data to make polynomial regression
     X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y_data, random_state = 1)
     polyr = LinearRegression().fit(X_train, y_train)

     this_score = polyr.score(X_test, y_test)
     if (this_score > polyr_degree):
          polyr_degree = this_score
          polyr_degree = this_degree


poly = PolynomialFeatures(degree=polyr_degree)
X_F1_poly = poly.fit_transform(OH_X_data)
# Prepare data to make polynomial regression
X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y_data, random_state = 1)
polyr = LinearRegression().fit(X_train, y_train)


# Get and save the r2 value of the model
polyr_scores = {
     'optimal_degree': poly_degree,
     'r2_train': polyr.score(X_train, y_train),
     'r2_test': polyr.score(X_test, y_test),
     'r2_cv': cross_val_score(polyr, X_train, y_train, cv=kf5, scoring="r2")
}

# Print r2 value
# print('R-squared score (training): {:.3f}'
#      .format(polr_scores['r2_train']))
# print('R-squared score (test): {:.3f}\n'
#      .format(polr_scores['r2_test']))
# print('R-squared score (test): {:.3f}\n'
#      .format(linr_scores['r2_cv'].mean()))




## Ridge Regression

In [166]:
# Import required libraries
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

# Prepare data 
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=735)


# for this_alpha in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40]:
ridgr_alpha = 0
ridgr_cv = 0
kf5 = KFold(n_splits=N, shuffle=True)
for this_a in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.35, 1.37, 1.43, 1.45]:
     ridgr = Ridge(alpha=this_a, random_state=1).fit(X_train, y_train)
     this_cv = cross_val_score(ridgr, X_train, y_train, cv=N, scoring='r2').mean()
     if (this_cv > ridgr_cv):
          ridgr_cv = this_cv
          ridgr_alpha = this_a

# Define final model with best alpha
ridgr = Ridge(alpha=ridgr_alpha, random_state=1).fit(X_train, y_train)

# Get and save r2 values of the model
ridgr_scores = {
     'optimal_alpha': ridgr_alpha,
     'r2_train': ridgr.score(X_train, y_train),
     'r2_test': ridgr.score(X_test, y_test),
     'r2_cv': cross_val_score(ridgr, X_train, y_train, cv=kf5, scoring="r2").mean()
}

for k, v in ridgr_scores.items():
     print(k + ": " + str(v))
# Print r2 values of the model
# print('R-squared score (training): {:.3f}'
#      .format(ridgr_scores['r2_train']))
# print('R-squared score (test): {:.3f}\n'
#      .format(ridgr_scores['r2_test']))
# print('R-squared score (test): {:.3f}\n'
#      .format(ridgr_scores['r2_cv'].mean()))

optimal_alpha: 1.37
r2_train: 0.862063615440508
r2_test: 0.8899703103623747
r2_cv: 0.8620229418077487


## Lasso Regression

In [117]:
# Import required libraries
from sklearn.linear_model import Lasso

# Prepare data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=1)

lassr_alpha = 0
lassr_cv = 0
kf5 = KFold(n_splits=N, shuffle=True)
for this_a in [0, 0.0001, 0.001, 0.01, 0.1]:
     lassr = Lasso(alpha=this_a, random_state=1).fit(X_train, y_train)
     this_cv = cross_val_score(ridgr, X_train, y_train, cv=5, scoring='r2').mean()
     if (this_cv > lassr_cv):
          lassr_cv = this_cv
          lassr_alpha = this_a



# Get and save r2 scores of the model
lassr_scores = {
     'optimal_alpha': lassr_alpha,
     'r2_train': lassr.score(X_train, y_train),
     'r2_test': lassr.score(X_test, y_test),
     'r2_cv': cross_val_score(lassr, X_train, y_train, cv=kf5, scoring="r2")
}

# Print r2 scores
# print('R-squared score (training): {:.3f}'
#      .format(lassr_scores['r2_train']))
# print('R-squared score (test): {:.3f}\n'
#      .format(lassr_scores['r2_test']))
# print('R-squared score (test): {:.3f}\n'
#      .format(lassr_scores['r2_cv'].mean()))

print(lassr_alpha)
print(lassr_cv)



C:\Users\talha\AppData\Local\Temp\ipykernel_11716\3812617348.py:11: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lassr = Lasso(alpha=this_a, random_state=1).fit(X_train, y_train)
c:\Users\talha\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\talha\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.363e+10, tolerance: 3.549e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model =

0
0.8696459614651998


## KNN

In [17]:
# Import required libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Prepare data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=0)

knn_neighbors = 1
# Define model
knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train.values.ravel())


kf5 = KFold(n_splits=N, shuffle=True)

# Get and save r2 scores for the model
knn_scores = {
     'optimal_nneighbors': knn_neighbors,
     'r2_train': knn.score(X_train, y_train),
     'r2_test': knn.score(X_test, y_test),
     'r2_cv': cross_val_score(lassr, X_train, y_train, cv=kf5, scoring="r2")
}

# Print r2 scores
print('R-squared score (training): {:.3f}'
     .format(knn_scores['r2_train']))
print('R-squared score (test): {:.3f}\n'
     .format(knn_scores['r2_test']))
print('R-squared score (test): {:.3f}\n'
     .format(knn_scores['r2_cv'].mean()))

R-squared score (training): 0.253
R-squared score (test): 0.026

R-squared score (test): 0.862



c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

## Linear SVM

In [102]:
# Import required libraries
from sklearn.svm import LinearSVC


X_train, X_test, y_train, y_test = train_test_split(OH_X_data_scaled, y_data.values.ravel(), random_state=735)

# Define and fit the model
linsvc = LinearSVC(C=10).fit(X_train, y_train)

kf5 = KFold(n_splits=N, shuffle=True)


linsvc_c = 1
linsvc_cv = 0
kf5 = KFold(n_splits=N, shuffle=True) 
for this_c in [2, 3, 4]:
    linsvc = LinearSVC(C=this_c, max_iter=9999).fit(X_train, y_train)
    this_cv = cross_val_score(linsvc, X_train, y_train, cv=kf5, scoring='r2').mean()
    if (this_cv > linsvc_cv):
        linsvc_cv = this_cv
        linsvc_c = this_c

# Get and save r2 scores for the model
linsvc_scores = {
    'optimal_c': linsvc_c,
    'r2_train': linsvc.score(X_train, y_train),
    'r2_test': linsvc.score(X_test, y_test),
    'r2_cv': cross_val_score(linsvc, X_train, y_train, cv=kf5, scoring="r2")
}



# Print r2 scores
# print('R-squared score (training): {:.3f}'
#      .format(knn_scores['r2_train']))
# print('R-squared score (test): {:.3f}\n'
#      .format(knn_scores['r2_test']))
# print('R-squared score (test): {:.3f}\n'
#      .format(knn_scores['r2_cv'].mean()))

print(linsvc_c)
print(linsvc_cv)

c:\Users\talha\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


3
0.6699856582194033


## Non-Linear SVM

In [156]:
# Import required libraries
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(OH_X_data_scaled, y_data.values.ravel())

svc_c = 0.1
svc = SVC(C=0.1, random_state=1, kernel="rbf", gamma="auto").fit(X_train, y_train)

kf5 = KFold(n_splits=N, shuffle=True)

svc_scores = {
    'optimal_c': svc_c,
    'r2_train': svc.score(X_train, y_train),
    'r2_test': svc.score(X_test, y_test),
    'r2_cv': cross_val_score(svc, X_train, y_train, cv=kf5, scoring='r2')
}
svc_c = 1
svc_cv = 0
kf5 = KFold(n_splits=N, shuffle=True) 
for this_c in [10, 1000, 10000]:
    svc = SVC(C=this_c).fit(X_train, y_train)
    this_cv = cross_val_score(svc, X_train, y_train, cv=kf5, scoring='r2').mean()
    if (this_cv > svc_cv):
        svc_cv = this_cv
        svc_c = this_c

print(svc_c)
print(svc_cv)

KeyboardInterrupt: 

## Create excel file to export scores

In [160]:
# Create excel file includeing scores etc
table = pd.DataFrame(columns=["R2 Train", "R2 Test", "R2 CV"])
table.loc["Linear Regression"] = [linr_scores["r2_train"], linr_scores["r2_test"], linr_scores["r2_cv"].mean()]
table.loc["Polynomial Regression"] = [polyr_scores["r2_train"], polyr_scores["r2_test"], polyr_scores["r2_cv"].mean()]
table.loc["Ridge Regression"] = [ridgr_scores["r2_train"], ridgr_scores["r2_test"], ridgr_scores["r2_cv"].mean()]
table.loc["Lasso Regression"] = [lassr_scores["r2_train"], lassr_scores["r2_test"], lassr_scores["r2_cv"].mean()]
table.loc["KNN"] = [knn_scores["r2_train"], knn_scores["r2_test"], knn_scores["r2_cv"].mean()]
table.loc["SVM (Non Linear)"] = [svc_scores["r2_train"], svc_scores["r2_test"], svc_scores["r2_cv"].mean()]
table.loc["SVM (Linear)"] = [linsvc_scores["r2_train"], linsvc_scores["r2_test"], linsvc_scores["r2_cv"].mean()]

table

,R2 Train,R2 Test,R2 CV
Linear Regression,0.862290,0.889609,0.864140
Polynomial Regression,0.866812,0.874708,0.868949
Ridge Regression,0.862064,0.889970,0.862808
Lasso Regression,0.866812,0.874707,0.868598
KNN,0.252874,0.026207,0.862297
SVM (Non Linear),0.012874,0.011034,-0.287620
SVM (Linear),0.084138,0.016552,0.661135


## Export Results

In [22]:
# Export Results
price_predictions = linr.predict(OH_X_data_without_prices)
price_predictions_knn = knn.predict(OH_X_data_without_prices)
# price_predictions_int = np.round_(price_predictions)
np.savetxt('linear.txt', price_predictions)
np.savetxt('knn.txt', price_predictions_knn)
# price_predictions_int



c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
